In [50]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [51]:
original_df=pd.read_csv('healthcare-dataset-stroke-data.csv')

In [52]:
original_df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


### Row dropping

In [53]:
df=original_df[original_df['gender']!='Other']
df.dropna(inplace=True)

C:\Users\ledif\AppData\Local\Temp\ipykernel_2148\3444059950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


### Dropping 'id' column

In [54]:
df.drop(columns='id', inplace=True)

C:\Users\ledif\AppData\Local\Temp\ipykernel_2148\1895964800.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns='id', inplace=True)


### New Columns with Dummies

In [55]:
Male=pd.get_dummies(df['gender'], drop_first=True, dtype=int)
Married=pd.get_dummies(df['ever_married'], drop_first=True, dtype=int)
Residence=pd.get_dummies(df['Residence_type'], drop_first=True, dtype=int)

### New OneHotEncoder Columns

In [56]:
print(df['work_type'].unique())
print(df['smoking_status'].unique())


['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']
['formerly smoked' 'never smoked' 'smokes' 'Unknown']


In [57]:
work_codes=['children','Never_worked','Govt_job','Private','Self-employed']
smoke_codes=['never smoked','Unknown','formerly smoked','smokes']

In [58]:
ohe=OneHotEncoder(handle_unknown='ignore',sparse_output=False, dtype=int)
ohe.set_output(transform='pandas')
Work_and_smoke=ohe.fit_transform(df[['work_type','smoking_status']])


In [59]:
le=LabelEncoder()
le.fit(work_codes)
df['work_type']=le.transform(df['work_type'])
le.fit(smoke_codes)
df['smoking_status']=le.transform(df['smoking_status'])

C:\Users\ledif\AppData\Local\Temp\ipykernel_2148\4033634226.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['work_type']=le.transform(df['work_type'])
C:\Users\ledif\AppData\Local\Temp\ipykernel_2148\4033634226.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['smoking_status']=le.transform(df['smoking_status'])


### Concatenating new columns and removing original ones

In [60]:
stroke_clean_df=pd.concat([df,Male],axis=1).drop(columns=['gender'])
stroke_clean_df['ever_married']=Married
stroke_clean_df=pd.concat([stroke_clean_df,Residence],axis=1).drop(columns=['Residence_type'])
# stroke_clean_df=pd.concat([stroke_clean_df,Work_and_smoke],axis=1).drop(columns=['work_type','smoking_status'])


In [61]:
stroke_clean_df.columns

Index(['age', 'hypertension', 'heart_disease', 'ever_married', 'work_type',
       'avg_glucose_level', 'bmi', 'smoking_status', 'stroke', 'Male',
       'Urban'],
      dtype='object')

### Moving stroke column to the end

In [62]:
stroke_clean_df=stroke_clean_df[['age', 'hypertension', 'heart_disease', 'ever_married', 'work_type',
       'avg_glucose_level', 'bmi', 'smoking_status', 'Male','Urban', 'stroke']]
stroke_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4908 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                4908 non-null   float64
 1   hypertension       4908 non-null   int64  
 2   heart_disease      4908 non-null   int64  
 3   ever_married       4908 non-null   int32  
 4   work_type          4908 non-null   int32  
 5   avg_glucose_level  4908 non-null   float64
 6   bmi                4908 non-null   float64
 7   smoking_status     4908 non-null   int32  
 8   Male               4908 non-null   int32  
 9   Urban              4908 non-null   int32  
 10  stroke             4908 non-null   int64  
dtypes: float64(3), int32(5), int64(3)
memory usage: 364.3 KB


In [63]:
stroke_clean_df.to_csv('clean-healthcare-categorical-dataset-stroke-data.csv',index=False)